In [1]:
import sys

In [2]:
TXT_PROJ_ROOT = "/home/gml/Documents/Master/M1_BIBS/S2/Structurale/"

In [3]:
sys.path.append(TXT_PROJ_ROOT)

In [39]:
from piestruct.utils.customobjs import (ObjDict as odict, Path as path)
from piestruct.utils.customfuncs import udir

In [5]:
import Bio.PDB as pdb
import freesasa
import nglview as nv

In [6]:
proj_root = path(TXT_PROJ_ROOT)

In [7]:
parse_pdb = (
    lambda x: 
    pdb
    .PDBParser(QUIET=True)
    .get_structure(id = x.name.replace(".pdb", ""), file = x.abs )
) # x doit être un fichier de type piestruct.utils.customobjs.Path

In [8]:
pdb_files = proj_root.joinpath("data").dglob("*.pdb")
pdb_files

{'1he8.pdb': PosixPath('/home/gml/Documents/Master/M1_BIBS/S2/Structurale/data/1he8.pdb'),
 '1HE8_AB_c.pdb': PosixPath('/home/gml/Documents/Master/M1_BIBS/S2/Structurale/data/1HE8_AB_c.pdb'),
 '1e8y.pdb': PosixPath('/home/gml/Documents/Master/M1_BIBS/S2/Structurale/data/1e8y.pdb'),
 '121p.pdb': PosixPath('/home/gml/Documents/Master/M1_BIBS/S2/Structurale/data/121p.pdb')}

In [9]:
pdb_files["121p.pdb"].name

'121p.pdb'

In [10]:
(biop_struct := odict({ 
        name.replace(".pdb", ""):  parse_pdb(location) 
        for name, location in pdb_files.items() 
}))

{'1he8': <Structure id=1he8>,
 '1HE8_AB_c': <Structure id=1HE8_AB_c>,
 '1e8y': <Structure id=1e8y>,
 '121p': <Structure id=121p>}

In [11]:
for struct in biop_struct.values():
    print(len(struct))

1
1
1
1


In [12]:
(biop_models := odict({
    name: struct[0] 
    for name, struct in biop_struct.items()
}))

{'1he8': <Model id=0>,
 '1HE8_AB_c': <Model id=0>,
 '1e8y': <Model id=0>,
 '121p': <Model id=0>}

## Créer les structures "freesasa"

In [13]:
structures = odict({ 
    name.replace(".pdb", ""): freesasa.Structure(location.abs) 
    for name, location in pdb_files.items() 
})
structures

{'1he8': <freesasa.Structure at 0x7efe43734870>,
 '1HE8_AB_c': <freesasa.Structure at 0x7efe43734120>,
 '1e8y': <freesasa.Structure at 0x7efe437349f0>,
 '121p': <freesasa.Structure at 0x7efe437349c0>}

## Calculer les aires

In [14]:
sasas = odict({
    name: freesasa.calc(struct)
    for name, struct in structures.items()
})
sasas

{'1he8': <freesasa.Result at 0x7efe424fd570>,
 '1HE8_AB_c': <freesasa.Result at 0x7efe424fd790>,
 '1e8y': <freesasa.Result at 0x7efe424fd610>,
 '121p': <freesasa.Result at 0x7efe424fd710>}

In [15]:
resid_areas = odict({
    name: sasa.residueAreas()
    for name, sasa in sasas.items()
})

In [16]:
monomere1 = resid_areas["121p"]
monomere2 = resid_areas["1e8y"]

In [17]:
monomere1.keys(), monomere2.keys()

(dict_keys(['A']), dict_keys(['A']))

In [18]:
for value in monomere1["A"].values():
    if value.relativeTotal > 1.0:
        print("Error")

In [19]:
relative_sasas = odict({
    name: odict({
        chain_name: sum(residu.relativeTotal for residu in chain.values())
        for chain_name, chain in entry.items()
    })
    for name, entry in resid_areas.items()
})

In [20]:
for entry, chains in relative_sasas.items():
    if len(chains) > 1:
        chains.update(odict({
            "total": sum(chain for chain in chains.values())
        }))

In [21]:
total_sasas = odict({
    name: odict({
        chain_name: sum(residu.total for residu in chain.values())
        for chain_name, chain in entry.items()
    })
    for name, entry in resid_areas.items()
})

In [22]:
for entry, chains in total_sasas.items():
    if len(chains) > 1:
        chains.update(odict({
            "total": sum(chain for chain in chains.values())
        }))

In [23]:
total_sasas

{'1he8': {'A': 32729.062149104324,
  'B': 7427.585501586975,
  'total': 40156.6476506913},
 '1HE8_AB_c': {'A': 32729.062149104324,
  'B': 7427.585501586975,
  'total': 40156.6476506913},
 '1e8y': {'A': 36797.43189172239},
 '121p': {'A': 8720.956397446842}}

In [24]:
relative_sasas

{'1he8': {'A': 201.56061472864042,
  'B': 45.29767600588998,
  'total': 246.85829073453039},
 '1HE8_AB_c': {'A': 201.56061472864042,
  'B': 45.29767600588998,
  'total': 246.85829073453039},
 '1e8y': {'A': 227.03229561572496},
 '121p': {'A': 53.29124167145358}}

In [25]:
views = odict({
    name: nv.show_biopython(model)
    for name, model in biop_models.items()
})

In [26]:
views["1HE8_AB_c"]

NGLWidget()

In [27]:
# Surface 
resid_areas["1HE8_AB_c"]["A"]["459"]
resid_areas["1HE8_AB_c"]["B"]["47"]

In [28]:
# Interface 
resid_areas["1HE8_AB_c"]["A"]["231"]
resid_areas["1HE8_AB_c"]["B"]["39"]

In [29]:
resid_areas["1HE8_AB_c"]["B"]["39"].relativeTotal

0.32488532749843224

In [30]:
resid_areas["1HE8_AB_c"]["B"]["39"].total

38.44692965616447

In [31]:
{ name: entry.keys() for name, entry in resid_areas.items() }

{'1he8': dict_keys(['A', 'B']),
 '1HE8_AB_c': dict_keys(['A', 'B']),
 '1e8y': dict_keys(['A']),
 '121p': dict_keys(['A'])}

In [32]:
sum(residu.relativeTotal for residu in monomere1["A"].values())

53.29124167145358

In [33]:
sum(sasa.total for sasa in monomere1["A"].values())

8720.956397446842

In [34]:
{
  i: {str(j): j**2 for j in range(8)}
  for i in list("abcd")
} 

{'a': {'0': 0, '1': 1, '2': 4, '3': 9, '4': 16, '5': 25, '6': 36, '7': 49},
 'b': {'0': 0, '1': 1, '2': 4, '3': 9, '4': 16, '5': 25, '6': 36, '7': 49},
 'c': {'0': 0, '1': 1, '2': 4, '3': 9, '4': 16, '5': 25, '6': 36, '7': 49},
 'd': {'0': 0, '1': 1, '2': 4, '3': 9, '4': 16, '5': 25, '6': 36, '7': 49}}

In [35]:
complexe = sasas['1HE8_AB_c'].residueAreas()

In [36]:
complexe.keys()

dict_keys(['A', 'B'])

In [40]:
udir(monomere1["A"]["1"])

['apolar',
 'hasRelativeAreas',
 'mainChain',
 'polar',
 'relativeApolar',
 'relativeMainChain',
 'relativePolar',
 'relativeSideChain',
 'relativeTotal',
 'residueNumber',
 'residueType',
 'sideChain',
 'total']